<a href="https://colab.research.google.com/github/olesyamba/Risk_analysis/blob/main/Practice_task_Portfolio_Risk_Hedg_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Для хранения, обработки данных и математических вычислений
import numpy as np
!pip install --upgrade pandas>=1.2
import pandas as pd
import scipy.stats as ss

# 1. Хеджирования в терминах чувствительностей
Дано: чувствительность доходности портфеля: 0.5 к S&P и -0.3 к нефти.

2 хеджирующих инструмента:

- фьючерс на S&P: чувствительность к S&P 1.04,
- фьючерс на нефть: чувствительность к индексу -0.1 и к нефти 0.95.

Найти структуру хеджирующего портфеля.

Цель: свести все чувствительности к 0.

- Чувствительность к S&P: $0.5 + 1.04\cdot x - 0.1 \cdot y = 0$

- Чувствительность к нефти: $-0.3 + 0.95 \cdot y = 0$


In [ ]:
y = 0.3/0.95
x = - (0.5 - 0.1*y)/1.04
print('Фьюч S&P {:.4f}, фьюч на нефть {:.4f}'.format(x, y))

# Чтобы найти стратегию хеджирования в штуках, нужно знать объем портфеля и цену фьючерсных контрактов.

Фьюч S&P -0.4504, фьюч на нефть 0.3158


# 2. Хеджирования в терминах чувствительностей
Дано: чувствительность цены портфеля в млн. долларов: 5 к S&P и -3 к нефти.

2 хеджирующих инструмента:
- фьючерс на S&P: чувствительность к S&P -- 104,000 долларов,
- фьючерс на нефть: чувствительность к индексу -10,000 долларов и к нефти 95,000 долларов.

Найти состав хеджирующего портфеля.

- Чувствительность к S&P: $5 + 0.104\cdot x - 0.01 \cdot y = 0$

- Чувствительность к нефти: $-3 + 0.095 \cdot y = 0$


In [ ]:
y = 3/0.095
x = - (5 - 0.01*y)/0.104
print('Количество фьючерсов S&P {:.4f}, количество фьючерсов на нефть {:.4f}'.format(x, y))

Количество фьючерсов S&P -45.0405, количество фьючерсов на нефть 31.5789


# 3. Соответствие ограничениям по риску

Текущий портфель имеет чувствительности в 15 млн и 20 млн к риск-факторам A и B.

Ковариационная матрица доходностей факторов A и B:

| |A|B|
|:-|:-:|:-:|
|A|0.01|0.005|
|B|0.005|0.03|

VaR на уровне 99% оценивают дельта-нормальным методом.

На VaR портфеля установлен лимит 5 млн. Захеджировать портфель инструментом с чувствительностями :
- к  А 1 млн,
- к В -2 млн соответственно,

так, чтобы уложиться в лимит.

Алгоритм:

- нужно ли это?
- это вообще возможно?
- если возможно, то как это сделать?

In [ ]:
# Ковариационная матрица
cov = pd.DataFrame({'A':[0.01, 0.005], 'B': [0.005, 0.03]}, index = ['A', 'B'])
print('Ковариационная матрица факторов\n', cov)

# Чувствительности исходного портфеля
s = pd.Series({'A':15, 'B': 20})
print('Вектор чувствительностей исходного портфеля\n', s)

sigma_prt = np.sqrt(s @ cov @ s.T) # сигма изменений стоимости портфеля
print('Сигма исходного портфеля:', sigma_prt)

level = .99
VaR_99 = sigma_prt*ss.norm.ppf(level)
print('VaR 99%', VaR_99)

Ковариационная матрица факторов
        A      B
A  0.010  0.005
B  0.005  0.030
Вектор чувствительностей исходного портфеля
 A    15
B    20
dtype: int64
Сигма исходного портфеля: 4.153311931459037
VaR 99% 9.66204838197819


- s - чувствительности исходного портфеля
- h - чувствительности хеджирующего инструмента
- $\Sigma$ - ковариационная матрица факторов
- $$s + x\cdot h$$ - чувствительности захеджированного портфеля как функция от объема хеджа ($x$)

Дисперсия захеджированного портфеля:
$$\sigma^2_p = (s + x\cdot h) \Sigma (s + x\cdot h)^T \to \min\limits_{x} $$

$$h\Sigma h^T x^2 + 2h \Sigma s x \to \min_{x}$$

$$2h\Sigma h^T x + 2h \Sigma s = 0 $$

$$x = - \frac{h \Sigma s}{h\Sigma h^T}$$

In [ ]:
# Чувствительности исходного портфеля
h = pd.Series({'A':1, 'B': -2})
print('Вектор чувствительностей хеджирующего инструмента\n', h)

x = - (h @ cov @ s)/(h @ cov @ h.T)
print('Объем хеджирования', x)

s_new = s + x*h
sigma_prt = np.sqrt(s_new @ cov @ s_new.T) # сигма изменений стоимости портфеля
print('Сигма исходного портфеля:', sigma_prt)

level = .99
VaR_99 = sigma_prt*ss.norm.ppf(level)
print('VaR 99%', VaR_99)

Вектор чувствительностей хеджирующего инструмента
 A    1
B   -2
dtype: int64
Объем хеджирования 10.0
Сигма исходного портфеля: 2.5
VaR 99% 5.815869685102102
